<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Create_Embeding_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import TokenTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from pyvi.ViTokenizer import tokenize
import os
from PyPDF2 import PdfReader
import docx

from Sentence_Processing import split_into_sentences, split_paragraph_sentence
import uuid
from chromadb.utils import embedding_functions
import pandas as pd

os.environ["WANDB_MODE"] ="offline"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = "auto"
model_id = r"D:\Tuan\ChatGPT\model\phobert-base-v2"
embedding_fun = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_id)
model = SentenceTransformer(model_id)

def pdf_to_text(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
        return text
def word_to_text(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        line =  paragraph.text.strip()
        if len(line)==0: continue # remove empty lines
        line = " ".join(line.split())  # removed duplicated space and \r \t \n
        text += line + "\n"
    return text

def _split_sentences(text):
    # Use regular expressions to split the text into sentences based on punctuation followed by whitespace.
    sentences = re.split(r'(?<=[.?!])\s+', text)
    return sentences
def _combine_sentences(sentences):
    # Create a buffer by combining each sentence with its previous and next sentence to provide a wider context.
    combined_sentences = []
    for i in range(len(sentences)):
        combined_sentence = sentences[i]
        if i > 0:
            combined_sentence = sentences[i-1] + ' ' + combined_sentence
        if i < len(sentences) - 1:
            combined_sentence += ' ' + sentences[i+1]
        combined_sentences.append(combined_sentence)
    return combined_sentences
def convert_to_vector(texts):
    # Try to generate embeddings for a list of texts using a pre-trained model and handle any exceptions.
    try:
        # response = openai.embeddings.create(
        #     input=texts,
        #     model="text-embedding-3-small"
        # )
        embeddings=[]
        for sen in texts:
            embeding = model.encode(sen)
            embeddings.append(embeding)
        embeddings = np.array(embeddings)
        return embeddings
    except Exception as e:
        print("An error occurred:", e)
        return np.array([])  # Return an empty array in case of an error
def _calculate_cosine_distances(embeddings):
    # Calculate the cosine distance (1 - cosine similarity) between consecutive embeddings.
    distances = []
    for i in range(len(embeddings) - 1):
        similarity = cosine_similarity([embeddings[i]], [embeddings[i + 1]])[0][0]
        distance = 1 - similarity
        distances.append(distance)
    return distances
def chunk_text(data_path):
    # Split the input text into individual sentences.
    single_sentences_list = split_sentences(data_path)
    # Combine adjacent sentences to form a context window around each sentence.
    combined_sentences = _combine_sentences(single_sentences_list)
    # Convert the combined sentences into vector representations using a neural network model.
    embeddings = convert_to_vector(combined_sentences)
    # Calculate the cosine distances between consecutive combined sentence embeddings to measure similarity.
    distances = _calculate_cosine_distances(embeddings)
    breakpoint_percentile_threshold = 50
    breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
    # Find all indices where the distance exceeds the calculated threshold, indicating a potential chunk breakpoint.
    indices_above_thresh = [i for i, distance in enumerate(distances) if distance > breakpoint_distance_threshold]
    # Initialize the list of chunks and a variable to track the start of the next chunk.
    chunks = []
    start_index = 0
    # Loop through the identified breakpoints and create chunks accordingly.
    for index in indices_above_thresh:
        chunk = ' '.join(single_sentences_list[start_index:index + 1])
        chunks.append(chunk)
        start_index = index + 1
    # If there are any sentences left after the last breakpoint, add them as the final chunk.
    if start_index < len(single_sentences_list):
        chunk = ' '.join(single_sentences_list[start_index:])
        chunks.append(chunk)
    # Return the list of text chunks.
    return chunks

def split_sentences(data_path):
    documents_list=[]
    for filename in os.listdir(data_path):
        print(f'Processing {filename}')
        index = 0
        ids_list = []
        text = ""
        if filename.endswith('.pdf'):  # pdf files
            text = pdf_to_text(os.path.join(data_path, filename))
        elif filename.endswith('.docx'):  # doc files
            text = word_to_text(os.path.join(data_path, filename))
        if text == "":
            continue  # ignore empty file
        sentences = split_into_sentences(text)
        for sen in sentences:
            documents_list.append(sen)
    return documents_list

def add_folder_chunk_by_sentence(path):
        documents_list = []
        ids_list = []

        for filename in os.listdir(path):
            print(f'Processing {filename} ')
            index = 0
            text=""
            if filename.endswith('.pdf'): #  pdf files
                text= pdf_to_text(os.path.join(path, filename))
            elif filename.endswith('.docx'): # doc files
                text = word_to_text(os.path.join(path, filename))
            if text=="":
                continue #ignore empty file

            sentences = split_into_sentences(text)
            for sen in sentences:
                documents_list.append(sen)
                ids_list.append(f"{filename}_{index}")
                index +=1
                print(f"{len(sen)} \n {sen}")
            #collection.add(documents=documents_list, ids=ids_list)
        return documents_list, ids_list

def semantic_search(question, num_chunks_distance=3, num_chunks=3):
    collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                                 embedding_function=embedding_fun)
    results = collection.query(
         query_texts=question,
         include=['distances', 'documents'],
         n_results=num_chunks,
    )
    # print("----------------")
    # print(results)
    # print("----------------")
    distance = results['distances'][0][0]
    ids = []
    for cur_chunk_id in results["ids"][0]:
        cur_chunk_id_index = int(cur_chunk_id.split('_')[-1])
        head_id = cur_chunk_id[0:len(cur_chunk_id) - len(str(cur_chunk_id_index))]
        for i in range(-num_chunks_distance, num_chunks_distance + 1, 1):
            ids.append(head_id + str(cur_chunk_id_index + i))

    ids = sorted(set(ids))
    results = results if len(ids) == 1 else collection.get(ids=ids)
    context = " ".join(s for s in results["documents"])
    return distance, context

def createDB():
    metadata_list = []
    collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                                 embedding_function=embedding_fun)
    documents_list, ids_list= add_folder_chunk_by_sentence(data_path)
    for i in range(len(documents_list)):
        metadata_list.append({"type":"common"})
    collection.add(documents=documents_list, ids=ids_list,metadatas=metadata_list)
    print("Done!")
def CreateData8Sys(file):
    df = pd.read_json(file)
    print(df)
    documents_list = []
    ids_list = []
    metadata_list = []
    for i, row in df.iterrows():
        id = row["ID"]
        ids_list.append(id)
        Items = row["Items"]
        documents_list.append(Items)
        Detail = row["Detail"]
        Image_Name = row["Image Name"]
        metadata_list.append({"type":"8system","content":f"{Detail}","image_path":f"{Image_Name}"})

    collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                                     embedding_function=embedding_fun)
    collection.add(documents=documents_list, ids=ids_list, metadatas=metadata_list)
    print("Done!")
def TestQA(question):
    collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                                 embedding_function=embedding_fun)
    results = collection.query(
        # query_embeddings=[model.encode(question).tolist()],
        query_texts=question,
        include=['distances', 'documents','metadatas'],
        n_results=5,
    )
    # #context =" ".join(s for s in results["documents"][0])
    print(results)


data_path = r"D:/Tuan/ChatGPT/SData"
#chunks = chunk_text(data_path)
client = chromadb.PersistentClient('QAVectorDB_Vi')

filepath =r'D:\Tuan\ChatGPT\data\8System.json'
# createDB()
# CreateData8Sys(filepath)
# collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
#                                                  embedding_function=embedding_fun)
# count = collection.count()
# print('--------------------------------------')
# print(count)

# question = "Tiêu chuẩn sử dụng băng dán Teflon phần nối của ống dẫn/ống khí nén như thế nào?"
# TestQA(question)


# kq = semantic_search(question)
# print(kq)



# question=["Samsung Display Vietnam (SDV) được thành lập khi nào?","CEO hiện tại của công ty Samsung Display là ai?"]
# answer =["Samsung Display Vietnam (SDV) được thành lập vào ngày 14 tháng 7 năm 2014.","CEO hiện tại của công ty Samsung Display là ông Choi Joo Sun."]

# question="CEO hiện tại của công ty Samsung Display là ai?"
# an1 ="Technology Innovation Team (Tech Inno Team) được thành lập từ năm 2018, là 1 trong các Team của nhà máy Samsung Display Việt Nam (SDV)."
# an2 ="Trụ sở chính của Samsung Display là tại Seoul (Hàn Quốc) và CEO hiện tại của công ty Samsung Display là ông Choi Joo Sun."
# emb1 = model.encode(question).tolist()
# emb2 = model.encode(an1).tolist()
# emb3 = model.encode(an2).tolist()
#
# print(emb1)
# print(emb2)
# print(emb3)
#
# similarity = cosine_similarity([emb1], [emb2])[0][0]
# distance1 = 1 - similarity
# similarity = cosine_similarity([emb1], [emb3])[0][0]
# distance2 = 1 - similarity
# print(distance1)
# print(distance2)





